In [0]:
class VectorSearch():
    def __init__(self):
        from databricks.vector_search.client import VectorSearchClient
        self.vs_client = VectorSearchClient(disable_notice=True)

    def get_vectors(self):
        ep_list = self.vs_client.list_endpoints()
        endpoint_names = [ep["name"] for ep in ep_list]
        return endpoint_names

    def get_indexes(self,endpoint_name):
        index_list = self.vs_client.list_indexes(endpoint_name)
        index_names = [index["name"] for index in index_list]
        return index_names

    def create_vector_endpoint(self, endpoint_name):
        endpoint_names = self.get_vectors()
        if len(endpoint_names) == 0 or endpoint_name not in endpoint_names:
          print(f"Creating Vector Endpoint...", end='')       
          #self.vs_client.create_endpoint_and_wait(name=endpoint_name,endpoint_type="STANDARD")
          self.vs_client.create_endpoint(name=endpoint_name,endpoint_type="STANDARD")
          print("Done")
        else:
          print(f"Vector Endpoint already exists")
        
    def delete_vector_endpoint(self, endpoint_name):
        #endpoint_names = self.get_vectors()
        ep_list = self.vs_client.list_endpoints()
        endpoint_names = [ep['name'] for ep in ep_list['endpoints']]
        if len(endpoint_names)>0 and endpoint_name in endpoint_names:
          print(f"Deleting Vector Endpoint...", end='')
          self.vs_client.delete_endpoint(name=endpoint_name)
          print("Done") 
        else:
          print(f"Vector Endpoint does not exist")

    def create_vector_index(self, index_name, source_table_name, 
                     primary_key, embedding_source_column, 
                     embedding_model_endpoint_name, vector_endpoint_name):
        indexes = self.get_indexes(vector_endpoint_name)
        if len(indexes)==0 or index_name in indexes:
          print(f"Creating Index...", end='')
          self.vs_client.create_delta_sync_index_and_wait(
              index_name=f"{index_name}",
              source_table_name=f"{source_table_name}",            
              primary_key=primary_key,
              embedding_source_column=embedding_source_column,
              embedding_model_endpoint_name=embedding_model_endpoint_name,
              pipeline_type="TRIGGERED",
              endpoint_name=vector_endpoint_name)
        else:
          print(f"Index already exists")
          print("Done")

    def sync_index(self, index_name):
        import time
        print(f"Syncing Index...", end='')
        index = self.vs_client.get_index(index_name=f"{index_name}")
        index.sync()
        index_status = False
        while index_status!=True:
            time.sleep(20)
            index_status = index.describe()["status"]["ready"] 
        print("Done")

    def delete_index(self, index_name,endpoint_name):
        #indexes = self.get_indexes(endpoint_name)
        index_list = self.vs_client.list_indexes(endpoint_name)
        indexes = [index["name"] for index in index_list["vector_indexes"]]
        if len(indexes)>0 and index_name in indexes:
          print(f"Deleting Index...", end='')
          self.vs_client.delete_index(index_name=f"{index_name}")
          print("Done")
        else:
          print(f"Index does not exist")

#VS = VectorSearch()